# Módulo de testeo para NMF

A continuación se harán pruebas para el archivo "Test_1.wav", que contiene la grabación de una pieza cuyos instrumentos son un violín y un piano. La idea es separarlos de manera independiente.

In [7]:
import numpy as np
import os
import matplotlib.pyplot as plt
import soundfile as sf
from sklearn.decomposition import NMF
from IPython.display import Audio
#from descriptor_functions import get_spectrogram, get_inverse_spectrogram
#from math_functions import wiener_filter

# Definición de algunas funciones previas

En primer lugar, se define la función que creará el espectrograma, en conjunto con la función de espectrograma inverso.

# Espectrograma

In [10]:
def get_spectrogram(audio, samplerate, N=512, padding=512, overlap=0,
                    window='tukey', spect_type='abs'):
    # Lista donde se almacenará los valores del espectrograma
    spect_mag = []
    spect_pha = []
    # Lista de tiempo
    times = []
    
    # Variables auxiliares
    t = 0   # Tiempo
    
    # Seleccionar ventana
    if window == 'tukey':
        wind_mask = tukey(N)
    elif window == 'hamming':
        wind_mask = hamming_window(N)
        
    # Seleccionar tipo de espectrograma
    if spect_type == 'abs':
        mag_func = lambda audio: 1/N * abs(audio) #** 2
    elif spect_type == 'dB':
        mag_func = lambda audio: 20*np.log10(abs(audio)*1/N) 
    
    # Iteración sobre el audio
    while audio.any():
        # Se corta la cantidad de muestras que se necesite, o bien, las que se
        # puedan cortar
        if len(audio) >= N:
            q_samples = N
            step = int(N * (1 - overlap))
        else:
            break
            # q_samples = step = len(audio)
            
        # Recorte en la cantidad de muestras
        audio_frame = audio[:q_samples]
        audio = audio[step:]
               
        # Ventaneando
        audio_frame_wind = audio_frame * wind_mask
        
        # Aplicando padding
        audio_padded = np.append(audio_frame_wind, [0] * padding)
        
        # Aplicando transformada de fourier
        audio_fft = np.fft.fft(audio_padded)
        
        # Calculando la magntiud y fase 
        mag = mag_func(audio_fft)
        pha = np.angle(audio_fft)
               
        # Agregando a los vectores del espectro
        spect_mag.append(mag[0:int((N+padding)/2)])
        spect_pha.append(pha[0:int((N+padding)/2)])
        
        # Agregando al vector de tiempo
        times.append(t)
        t += step/samplerate
        
    # Generar el vector de frecuencias para cada ventana
    freqs = np.linspace(0, samplerate/2, (N+padding)/2)
        
    # Una vez obtenido el spect_mag y spect_pha, se pasa a matriz
    spect_mag = np.array(spect_mag)
    spect_pha = np.array(spect_pha)
    
    # Se retornan los valores que permiten construir el espectrograma correspondiente
    return times, freqs, spect_mag.T, spect_pha.T

# Espectrograma inverso

In [ ]:
def get_inverse_spectrogram(X, window='tukey'):
    # Obtener la dimensión de la matriz
    rows, cols = X.shape
    
    # Seleccionar ventana
    if window == 'tukey':
        wind_mask = tukey(rows)
    elif window == 'hamming':
        wind_mask = hamming_window(rows) 
    
    # Definición de una lista en la que se almacena la transformada inversa
    inv_spect = []
    
    # Transformando columna a columna
    for i in range(cols):
        inv_spect += list(wind_mask * np.fft.ifft(X[:, i]))
        
    return inv_spect

# Trabajo con el audio

En primer lugar, se obtendrá el espectrograma para corroborar que la función implementada tiene la misma forma que el ejemplo de la página de MIR.

En primer lugar, se hace lectura del audio, y dado que viene en formato estéreo, se mezclan ambos para obtener la señal entre -1 y 1 con el objetivo de que no se sature el sonido.

In [13]:
# Lectura del audio
audio, samplerate = sf.read('Test_1.wav')

# Normalizar audio para ambos canales
audio[:,0] = audio[:,0]/max(abs(audio[:,0])) 
audio[:,1] = audio[:,1]/max(abs(audio[:,1]))

# Mezclando ambos canales para obtener una señal monoaural
audio_mono = 0.5 * (audio[:,0] + audio[:,1])

# Escuchar
Audio(audio_mono, rate=samplerate)

# Definición de parámetros de trabajo

In [ ]:
# Opciones del script
comps_list = [2] #range(10, 500, 20)
N = 256     # Cantidad de muestras por cada frame

In [ ]:
# Aplicando la función del espectrograma

# Obteniendo el espectrograma
t, f, v, phase = get_spectrogram(audio_mono, samplerate, N=N, padding=N,
                                 plot=False, spect_type='dB')

# Guardar valores de la dimensión del espectrograma
row_dim, col_dim = v.shape